In [9]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks


import warnings
warnings.filterwarnings("ignore")

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=1,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=1,
)


# Retrieve features from feature groups
player_stats = player_stats_fg.select_all()
game_events = game_events_fg.select_all()

# Explicitly prefix column names to avoid ambiguity
player_stats = player_stats.with_column_renamed("player_name", "player_stats_player_name")
game_events = game_events.with_column_renamed("player_name", "game_events_player_name")

# Join the feature groups on the prefixed columns
selected_features = player_stats.join(
    game_events,
    on=["player_stats_player_name", "game_events_player_name"]
)

# Display the first 10 rows of the joined features
selected_features.show(10)


feature_view = fs.get_or_create_feature_view(
    name='clash_royale_fv',
    description="Clash Royale game prediction features",
    version=1,
    labels=['game_outcome'],
    query=selected_features,
)

2025-01-05 19:50:24,429 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 19:50:24,434 INFO: Initializing external client
2025-01-05 19:50:24,435 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 19:50:25,786 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700


AttributeError: 'Query' object has no attribute 'with_column_renamed'. 

In [11]:
player_stats_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 


,player_name,deck,result
0,حسین شهبازی,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",WIN
1,Diwel26,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",LOSE


In [12]:
game_events_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (33.21s) 


,player_name,deck,result
0,#PLQ0YQLLQ,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
1,#2CPCL9PYY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
2,#9RJ0J9RRR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
3,#2L9UJQJG2,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",WIN
4,#JP8CV0LL,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
...,...,...,...
129604,#8GLLR99G0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
129605,#PRQUQL9YY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
129606,#8VPLJ2V2P,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",LOSE
129607,#UQ0QQRC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",LOSE


In [ ]:

start_date_test_data = "2024-12-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

# Split data into train and test sets
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)

# Drop unnecessary columns
train_features = X_train.drop(['date', 'player_id'], axis=1)
test_features = X_test.drop(['date', 'player_id'], axis=1)

# Create and train the XGBoost Regressor
xgb_regressor = XGBRegressor()
xgb_regressor.fit(train_features, y_train)

# Predict game outcomes on the test set
y_pred = xgb_regressor.predict(test_features)

# Calculate and print metrics
mse = mean_squared_error(y_test.iloc[:, 0], y_pred)
print("MSE:", mse)

r2 = r2_score(y_test.iloc[:, 0], y_pred)
print("R squared:", r2)

# Save predictions to DataFrame
df = y_test
df['predicted_game_outcome'] = y_pred

# Create directory for model artifacts
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Visualize and save feature importance
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define input and output schemas
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_dict = model_schema.to_dict()

# Save the XGBoost model
xgb_regressor.save_model(model_dir + "/model.json")
mr = project.get_model_registry()

# Register the model in the Hopsworks model registry
cr_model = mr.python.create_model(
    name="clash_royale_xgboost_model",
    metrics={"mse": mse, "r2": r2},
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="Clash Royale game outcome predictor",
)

# Save the model artifacts to the registry
cr_model.save(model_dir)
